- **Name:** 11.2_sql_joins
- **Author:** Shamas Imran
- **Desciption:** SQL-style joins using Spark SQL queries
- **Date:** 19-Aug-2025
<!--
REVISION HISTORY
Version          Date        Author           Desciption
01           19-Aug-2025   Shamas Imran       Executed inner, left, right joins in SQL  
                                              Registered DataFrames as temp views  
                                              Verified SQL join outputs  
-->

In [0]:

from pyspark.sql.types import *
from datetime import date


student_schema = StructType([
    StructField('StudentID', IntegerType(), False),
    StructField('StudentName', StringType(), True),
    StructField('StudentAge', IntegerType(), True)
])

student_data = [
        (1, "Alice", 34), 
        (2, "Bob", 45), 
        (3, "Charlie", 29),
        (4, "Shamas", 40)
        ]

df_student = spark.createDataFrame(student_data, student_schema)


course_schema = StructType([
    StructField('CourseID', IntegerType(), False),
    StructField('CourseName', StringType(), True),
    StructField('CourseTitle', StringType(), True),
])

course_data = [
        (1, "Physics", "1111"), 
        (2, "Chemistry", "2222"), 
        (3, "English", "3333"),
        (4, "Computer Science", "4444")
        ]

df_course = spark.createDataFrame(course_data, course_schema)


# Enrollment schema
enrollment_schema = StructType([
    StructField("EnrollmentID", IntegerType(), False),
    StructField("StudentID_FK", IntegerType(), False),
    StructField("CourseID_FK", IntegerType(), False),
    StructField("EnrollmentDate", DateType(), True)
])

enrollment_data = [
    (1, 1, 1, date(2023, 9, 1)),   # Alice -> Physics
    (2, 2, 2, date(2023, 9, 2)),   # Bob -> Chemistry
    (3, 4, 4, date(2023, 9, 4)),   # Shamas -> Computer Science
    (4, 1, 2, date(2023, 9, 5)),   # Alice -> Chemistry
]

df_enrollment = spark.createDataFrame(enrollment_data, enrollment_schema)


In [0]:
# Create temp views for SQL queries
df_student.createOrReplaceTempView("student_temp")
df_course.createOrReplaceTempView("course_temp")
df_enrollment.createOrReplaceTempView("enrollment_temp")

In [0]:
%sql
-- INNER JOIN: returns only students who are enrolled in a course, along with course details
SELECT e.EnrollmentID, s.StudentName, c.CourseName, e.EnrollmentDate
FROM enrollment_temp e
INNER JOIN student_temp s
    ON e.StudentID_FK = s.StudentID
INNER JOIN course_temp c
    ON e.CourseID_FK = c.CourseID;


In [0]:
%sql
-- LEFT JOIN: returns all students and their enrollments if available; students with no enrollment will show null for course
SELECT s.StudentName, c.CourseName, e.EnrollmentDate
FROM student_temp s
LEFT JOIN enrollment_temp e
    ON s.StudentID = e.StudentID_FK
LEFT JOIN course_temp c
    ON e.CourseID_FK = c.CourseID;

In [0]:
%sql
-- RIGHT JOIN: returns all enrollments; if a student is missing, StudentName will be null (hypothetical case)
SELECT e.EnrollmentID, s.StudentName, c.CourseName, e.EnrollmentDate
FROM student_temp s
RIGHT JOIN enrollment_temp e
    ON s.StudentID = e.StudentID_FK
RIGHT JOIN course_temp c
    ON e.CourseID_FK = c.CourseID;


In [0]:
%sql
-- FULL OUTER JOIN: returns all students and all enrollments, matching where possible; unmatched rows from either side show null
SELECT s.StudentName, e.EnrollmentID, e.EnrollmentDate
FROM student_temp s
FULL OUTER JOIN enrollment_temp e
    ON s.StudentID = e.StudentID_FK;


In [0]:
%sql
-- CROSS JOIN: every student paired with every course (Cartesian product)
SELECT s.StudentName, c.CourseName
FROM student_temp s
CROSS JOIN course_temp c;


In [0]:
%sql
-- LEFT SEMI JOIN: returns students who have at least one enrollment; only columns from left (student) are retained
SELECT s.StudentID, s.StudentName, s.StudentAge
FROM student_temp s
WHERE EXISTS (
    SELECT 1
    FROM enrollment_temp e
    WHERE s.StudentID = e.StudentID_FK
);


In [0]:
%sql
-- LEFT ANTI JOIN: returns students who are not enrolled in any course
SELECT s.StudentID, s.StudentName, s.StudentAge
FROM student_temp s
WHERE NOT EXISTS (
    SELECT 1
    FROM enrollment_temp e
    WHERE s.StudentID = e.StudentID_FK
);

In [0]:
spark.catalog.dropTempView("student_temp")
spark.catalog.dropTempView("course_temp")
spark.catalog.dropTempView("enrollment_temp")